In [2]:
# 🧠 Core libraries
!pip install pandas gradio ollama

# 📊 For Excel file support
!pip install openpyxl xlrd

# 📄 For PDF file support
!pip install pdfplumber


In [3]:
import ollama
import pandas as pd
import gradio as gr
import pdfplumber

# ✅ AI Insight Generator
def generate_insights(df_summary, user_prompt):
    prompt = f"{user_prompt}\n\nDataset Preview:\n{df_summary.head().to_string(index=False)}"
    response = ollama.chat(
        model="llama3.2",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['message']['content']

# ✅ Handle Different File Types
def read_file(file):
    filename = file.name.lower()
    if filename.endswith(".csv"):
        df = pd.read_csv(file.name)
    elif filename.endswith(".xlsx") or filename.endswith(".xls"):
        df = pd.read_excel(file.name)
    elif filename.endswith(".pdf"):
        text_content = ""
        with pdfplumber.open(file.name) as pdf:
            for page in pdf.pages:
                text_content += page.extract_text() or ""
        # Create a DataFrame with one column of extracted text for uniform processing
        df = pd.DataFrame({"text": [text_content]})
    else:
        raise ValueError("Unsupported file type. Please upload CSV, Excel, or PDF.")
    return df

# ✅ EDA Function
def eda_analysis(file, user_prompt):
    df = read_file(file)
    insights = generate_insights(df, user_prompt)
    return insights

# ✅ Gradio Interface
demo = gr.Interface(
    fn=eda_analysis,
    inputs=[
        gr.File(label="📂 Upload File (CSV, Excel, or PDF)"),
        gr.Textbox(label="💬 Custom Prompt", placeholder="e.g. Give me the tools in the dataset")
    ],
    outputs="text",
    title="🧠 AI-Powered EDA with LLaMA",
    description="Upload a CSV, Excel, or PDF file and enter your own prompt to get AI-generated insights."
)

# ✅ Launch App
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7861
IMPORTANT: You are using gradio version 3.41.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://461162a46fdddcbb99.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
